### Donwload Data

In [ ]:
import requests
from urllib.parse import urlencode


def download(public_key: str, file_name: str):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']

    # Загружаем файл и сохраняем его
    download_response = requests.get(download_url)
    with open(file_name, 'wb') as f:   # Здесь укажите нужный путь к файлу
        f.write(download_response.content)

In [ ]:
download('https://disk.yandex.ru/d/MR3TQAMSWcbRTA', 'train.csv')

In [ ]:
download('https://disk.yandex.ru/d/0rIpN8xRGkqHDA', 'test.csv')

### Main Code

#### 1

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Загрузка данных
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Создание словарей соответствий
ru_en_dict = {
    'й': 'q', 'ц': 'w', 'у': 'e', 'к': 'r', 'е': 't', 'н': 'y', 'г': 'u', 'ш': 'i', 'щ': 'o', 'з': 'p', 'х': '[', 'ъ': ']',
    'ф': 'a', 'ы': 's', 'в': 'd', 'а': 'f', 'п': 'g', 'р': 'h', 'о': 'j', 'л': 'k', 'д': 'l', 'ж': ';', 'э': "'",
    'я': 'z', 'ч': 'x', 'с': 'c', 'м': 'v', 'и': 'b', 'т': 'n', 'ь': 'm', 'б': ',', 'ю': '.', '.': '/',
    'ё': '`'
}
en_ru_dict = {v: k for k, v in ru_en_dict.items()}

def switch_layout(word):
    """Переключает раскладку слова."""
    if any(c in ru_en_dict for c in word):
        return ''.join(ru_en_dict.get(c, c) for c in word)
    else:
        return ''.join(en_ru_dict.get(c, c) for c in word)

# Подготовка данных для обучения
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
X = vectorizer.fit_transform(train_data['text'])
y = (train_data['text'] != train_data['label']).astype(int)

# Обучение модели
model = MultinomialNB()
model.fit(X, y)

def find_and_fix_error(text):
    """Находит и исправляет ошибочное слово в тексте."""
    words = text.split()
    X_test = vectorizer.transform([text])
    if model.predict(X_test)[0] == 1:  # Если модель предсказывает наличие ошибки
        for i, word in enumerate(words):
            switched_word = switch_layout(word)
            if switched_word != word:
                new_text = ' '.join(words[:i] + [switched_word] + words[i+1:])
                if model.predict(vectorizer.transform([new_text]))[0] == 0:
                    return new_text
    return text  # Если ошибка не найдена или модель не предсказала ошибку

# Применяем функцию к тестовым данным
test_data['corrected'] = test_data['text'].apply(find_and_fix_error)

# Сохраняем результаты
test_data['corrected'].to_csv('predictions.csv', index=False, header=False)

print("Предсказания сохранены в файл 'predictions.csv'")

# Проверка точности на

Предсказания сохранены в файл 'predictions.csv'


In [ ]:
pred = train_data['text'].apply(find_and_fix_error)

In [ ]:
(train_data['label'] == pred).sum()

8389

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18960 entries, 0 to 18959
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18960 non-null  object
 1   label   18960 non-null  object
dtypes: object(2)
memory usage: 296.4+ KB


In [ ]:
set(en_ru_dict) & set(ru_en_dict)

{'.'}

#### 2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Загрузка данных
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Создание словарей соответствий
ru_en_dict = {
    'й': 'q', 'ц': 'w', 'у': 'e', 'к': 'r', 'е': 't', 'н': 'y', 'г': 'u', 'ш': 'i', 'щ': 'o', 'з': 'p', 'х': '[', 'ъ': ']',
    'ф': 'a', 'ы': 's', 'в': 'd', 'а': 'f', 'п': 'g', 'р': 'h', 'о': 'j', 'л': 'k', 'д': 'l', 'ж': ';', 'э': "'",
    'я': 'z', 'ч': 'x', 'с': 'c', 'м': 'v', 'и': 'b', 'т': 'n', 'ь': 'm', 'б': ',', 'ю': '.',
    'ё': '`'
}
en_ru_dict = {v: k for k, v in ru_en_dict.items()}

def switch_layout(word):
    """Переключает раскладку слова."""
    return ''.join(ru_en_dict.get(c, en_ru_dict.get(c, c)) for c in word)

def generate_variants(text):
    """Генерирует все варианты текста с одним замененным словом."""
    words = text.split()
    variants = [text]  # Добавляем исходный текст
    for i, word in enumerate(words):
        switched_word = switch_layout(word)
        if switched_word != word:
            new_text = ' '.join(words[:i] + [switched_word] + words[i+1:])
            variants.append(new_text)
    return variants

# Подготовка данных для обучения модели
X = train_data['label'].tolist() + train_data['text'].tolist()
y = [1] * len(train_data) + [0] * len(train_data)

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_vec, y_train)

# Оценка модели на валидационной выборке
val_accuracy = model.score(X_val_vec, y_val)
print(f"Точность модели на валидационной выборке: {val_accuracy:.2f}")

def find_correct_variant(text):
    """Находит наиболее вероятный правильный вариант текста."""
    variants = generate_variants(text)
    variant_vectors = vectorizer.transform(variants)
    probabilities = model.predict_proba(variant_vectors)[:, 1]  # Вероятности для класса 1 (правильный текст)
    best_variant_index = np.argmax(probabilities)
    return variants[best_variant_index]

# Применяем функцию к тестовым данным
test_data['corrected'] = test_data['text'].apply(find_correct_variant)

# Сохраняем результаты
test_data['corrected'].to_csv('predictions.csv', index=False, header=False)

print("Предсказания сохранены в файл 'predictions.csv'")

# Проверка точности на тренировочных данных
train_data['predicted'] = train_data['text'].apply(find_correct_variant)
accuracy = (train_data['predicted'] == train_data['label']).mean()
print(f"Точность на тренировочных данных: {accuracy:.2f}")

Точность модели на валидационной выборке: 0.26
Предсказания сохранены в файл 'predictions.csv'
Точность на тренировочных данных: 0.50


#### 3

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
print(train.head().to_string())

                                                                                                         text                                                                                                       label
0                                                                    и старательности полкового rjvfylbhf был                                                                    и старательности полкового командира был
1                                                 о котором старались умолчать tlbycndtyyj потому, что другая                                                 о котором старались умолчать единственно потому, что другая
2                                                                                      или пуля dfc не брала,                                                                                      или пуля вас не брала,
3  друг друга, нестройною, но веселою b оживленною толпой побежали наши под гору за расстроенными французами.  друг друга, нестр

In [ ]:
import pandas as pd

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test['text'].apply(len).max()

139

In [ ]:
test['text'].apply(lambda x: len(x.split())).max()

15

In [ ]:
print(test.head().to_string())

                                                                                   text
0  петруша, и их любовников, – сказал он. пьер, опустив глаза, пил из своего бокала, не
1                                               мог не привлечь его внимания. наполеон,
2                                                                 деньги на карты, а то
3           у себя во франции на такие пустяки не j,hfoftv внимания. государь ничего не
4                       – откуда и докуда, ваше величество? – от l.htyintqyf до кремса.


In [ ]:
train.shape

(18960, 2)

In [ ]:
test.shape

(4740, 1)

In [ ]:
4740 / 18960

0.25

In [ ]:
for i in range(5):
    print(train['text'][i])
print('-'*10)
for i in range(5):
    print(train['label'][i])

и старательности полкового rjvfylbhf был
о котором старались умолчать tlbycndtyyj потому, что другая
или пуля dfc не брала,
друг друга, нестройною, но веселою b оживленною толпой побежали наши под гору за расстроенными французами.
берейторами в расшитых попонах красивые запасные царские лошади. как будто
----------
и старательности полкового командира был
о котором старались умолчать единственно потому, что другая
или пуля вас не брала,
друг друга, нестройною, но веселою и оживленною толпой побежали наши под гору за расстроенными французами.
берейторами в расшитых попонах красивые запасные царские лошади. как будто


In [ ]:
for i in range(5):
    print(test['text'][i])

петруша, и их любовников, – сказал он. пьер, опустив глаза, пил из своего бокала, не
мог не привлечь его внимания. наполеон,
деньги на карты, а то
у себя во франции на такие пустяки не j,hfoftv внимания. государь ничего не
– откуда и докуда, ваше величество? – от l.htyintqyf до кремса.


#### 4

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')

In [ ]:
sum(train['text'] == train['label'])

7985

In [ ]:
train.shape

(18960, 2)

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test['text'].apply(len).count()

4740

In [ ]:
print(train.head().to_string())

                                                                                                         text                                                                                                       label
0                                                                    и старательности полкового rjvfylbhf был                                                                    и старательности полкового командира был
1                                                 о котором старались умолчать tlbycndtyyj потому, что другая                                                 о котором старались умолчать единственно потому, что другая
2                                                                                      или пуля dfc не брала,                                                                                      или пуля вас не брала,
3  друг друга, нестройною, но веселою b оживленною толпой побежали наши под гору за расстроенными французами.  друг друга, нестр

#### 5

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
def calc_space(row: str) -> int:
    now = 0
    best = 0
    for le in row:
        if le == ' ':
            now += 1
        else:
            now = 0
        best = max(best, now)

    return best

In [ ]:
train['label'].apply(calc_space).max()

1

In [ ]:
train['text'].apply(calc_space).max()

1

In [ ]:
test['text'].apply(calc_space).max()

1

### 6

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.text.apply(len).max()

133

In [ ]:
print(1)

1


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# Функция для генерации вариантов
def generate_variants(text):
    variants = set([text])
    words = text.split()

    ru_layout = "йцукенгшщзхъфывапролджэячсмитьбю.ё"
    en_layout = "qwertyuiop[]asdfghjkl;'zxcvbnm,./`"
    layout_map = dict(zip(ru_layout, en_layout) + zip(en_layout, ru_layout))

    for i, word in enumerate(words):
        switched_word = ''.join(layout_map.get(c, c) for c in word)
        if switched_word != word:
            new_text = ' '.join(words[:i] + [switched_word] + words[i+1:])
            variants.add(new_text)

    return variants

# Загрузка и подготовка данных
df = pd.read_csv('train.csv')
texts = df['text'].tolist()
labels = df['label'].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Инициализация токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Создание кастомного датасета
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = 1 if self.texts[idx] != self.labels[idx] else 0

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Создание датасетов
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

# Инициализация модели
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Инициализация тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Обучение модели
trainer.train()

# Функция для исправления текста
def correct_text(text, model, tokenizer):
    variants = generate_variants(text)

    inputs = tokenizer(list(variants), return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)

    probs = torch.softmax(outputs.logits, dim=1)
    original_prob = probs[0][0].item()

    best_variant = text
    best_prob = original_prob

    for i, variant in enumerate(variants):
        if variant != text:
            variant_prob = probs[i][1].item()
            if variant_prob > best_prob:
                best_variant = variant
                best_prob = variant_prob

    return best_variant

# Обработка тестового файла
test_df = pd.read_csv('test.csv')
test_texts = test_df['text'].tolist()

corrected_texts = []
for text in test_texts:
    corrected_text = correct_text(text, model, tokenizer)
    corrected_texts.append(corrected_text)

# Запись результатов в файл
with open('output.txt', 'w', encoding='utf-8') as f:
    for text in corrected_texts:
        f.write(text + '\n')

### 7

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

,text,label
0,и старательности полкового rjvfylbhf был,и старательности полкового командира был
1,о котором старались умолчать tlbycndtyyj потом...,о котором старались умолчать единственно потом...
2,"или пуля dfc не брала,","или пуля вас не брала,"
3,"друг друга, нестройною, но веселою b оживленно...","друг друга, нестройною, но веселою и оживленно..."
4,берейторами в расшитых попонах красивые запасн...,берейторами в расшитых попонах красивые запасн...


In [ ]:
test.head()

,text
0,кутузов? – быстро проговорил приезжий генерал ...
1,ценит он друга и как много ждет от него d
2,"(видно, передние остановились), и пронесся слу..."
3,– обратились вдруг офицеры r вошедшему. – похо...
4,"сила не может низвергнуть, есть сохранение и п..."


In [ ]:
res = set()
for text in train.text:
    for s in text:
        if not s.islower():
            res.add(s)

In [ ]:
res

{' ',
 '!',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 '[',
 ']',
 '`',
 '«',
 '»',
 '–',
 '—',
 '’',
 '“',
 '„',
 '…'}

In [ ]:
ru_to_en = {
    'й':'q', 'ц':'w', 'у':'e', 'к':'r', 'е':'t', 'н':'y', 'г':'u', 'ш':'i', 'щ':'o', 'з':'p',
    'ф':'a', 'ы':'s', 'в':'d', 'а':'f', 'п':'g', 'р':'h', 'о':'j', 'л':'k', 'д':'l', 'ж': ';',
    'я':'z', 'ч':'x', 'с':'c', 'м':'v', 'и':'b', 'т':'n', 'ь':'m',
    'б':',', 'ю':'.', 'ё':'`'
}

en_to_ru = {v: k for k, v in ru_to_en.items()}

In [ ]:
ru_set = set(ru_to_en)
en_set = set(en_to_ru)

def maybe_change(row: str) -> str:
    line = row.split()
    n = len(line)
    mask = [0] * n

    for i in range(n):
        now = set(line[i])
        fi = len(ru_set | now)
        se = len(en_set | now)
        if fi > se:
            mask[i] += 1
        elif se > fi:
            mask[i] -= 1

    for i in range(n):
        l, m, r = i - 1, i, (i + 1) % n
        if mask[m] == 0:
            continue
        if mask[l] == mask[r]  and mask[m] != mask[l]:
            if mask[m] == 1:
                line[i] = ''.join(en_to_ru[s] if s in en_to_ru else s for s in line[i])
            else:
                line[i] = ''.join(ru_to_en[s] if s in ru_to_en else s for s in line[i])
            return ' '.join(line)

    return row

In [ ]:
(train.text.apply(maybe_change) == train.label).sum() / train.shape[0]

0.9021624472573839

In [ ]:
subm = test.text.apply(maybe_change)

In [ ]:
subm.to_csv('submit.csv', header=False, index=False, sep='%', quoting=False)